In [ ]:
import pickle
import pandas as pd

with open('Strength_of_Emotion.pkl', 'rb') as xx:
    Strength_of_Emotion = pickle.load(xx)



with open('Sentiment_Score.pkl', 'rb') as f:
    Sentiment_Score = pickle.load(f)
    



with open('errors_index.pkl', 'rb') as f:
    errors_index = pickle.load(f)


df = pd.read_csv('tweets_lemmatized_new.csv')
df = df.drop(errors_index)

df['Date'] = pd.to_datetime(df['Date'])
df['Sentiment_Score'] = Sentiment_Score
df['Strength_of_Emotion'] = Strength_of_Emotion
df['Weighted_Sentiment_Score'] = df['Sentiment_Score'] * df['Strength_of_Emotion']
df = df.iloc[::-1]

######################resample the timeseries data in one minute frequency 

# Set 'date' column as index
df.set_index('Date', inplace=True)

# Resample the data frame to one minute frequency and count the number of tweets for each minute
count = df['Tweet'].resample('1T').count()

# Resample the data frame to one minute frequency and sum the Sentiment_Score for each minute
sum_score = df['Sentiment_Score'].resample('1T').sum()

sum_weighted_score = df['Weighted_Sentiment_Score'].resample('1T').sum()


# Combine the count and sum_score data frames into a new data frame
result = pd.concat([count, sum_score, sum_weighted_score], axis=1)

# Fill null values with 0
result.fillna(0, inplace=True)

# Reset the index and rename the columns
result = result.reset_index().rename(columns={'Tweet': 'num_tweets', 'Sentiment_Score': 'sum_score'})


#######################

result.set_index('Date', inplace=True)

# create a new dataframe with the desired date range
start_date = pd.to_datetime('2019-03-04 00:00:00')
end_date = pd.to_datetime('2019-03-04 00:02:00') # get the first index value from the original dataframe
new_index = pd.date_range(start=start_date, end=end_date, freq='1min')

start_date_last = pd.to_datetime('2019-12-10 23:59:00')
#end_date_last = pd.to_datetime('2019-12-11 00:00:00')
new_index_last = pd.date_range(start=start_date_last, end=start_date_last, freq='1min')

new_df = pd.DataFrame(index=new_index, columns=result.columns)

new_df_last = pd.DataFrame(index=new_index_last, columns=result.columns)

# merge the two dataframes
final_df = pd.concat([new_df , result, new_df_last], axis=0)

# reset the index to a regular column
final_df.reset_index(inplace=True)
final_df.fillna(0, inplace=True)
final_df.rename(columns={'index': 'date'}, inplace=True)

final_df.to_csv('The features of twitter.csv')
final_df.tail()



,date,num_tweets,sum_score,Weighted_Sentiment_Score
406075,2019-12-10 23:55:00+00:00,0,0.0,0.00
406076,2019-12-10 23:56:00+00:00,0,0.0,0.00
406077,2019-12-10 23:57:00+00:00,1,-0.1,-0.01
406078,2019-12-10 23:58:00+00:00,1,0.1,0.01
406079,2019-12-10 23:59:00,0,0.0,0.00
